In [1]:
from ipynb.fs.full.setup import *
from ipynb.fs.full.A_matrix import *
from ipynb.fs.full.boundaryCellDisplacement import *
from ipynb.fs.full.boundaryCellTraction import *
from ipynb.fs.full.boundaryPointTraction import *
from ipynb.fs.full.cell_corner_BCs import *
from ipynb.fs.full.boundary_conditions import *

import math
from matplotlib import pyplot as plt


40
4
40
4
Shear Modulus (mu): 76923076923.07692
Lame Modulus (Lambda): 115384615384.61539


In [2]:
## Time loop

# ! mkdir savedVariables

for time in t:

    print("\nTime = " + str(time))

    # Set number of maximum iterations for convergence

    icorr = 1
    maxcorr = 100000
    fac = 10 ** math.ceil((math.log((A("x").a_P), 10)))

    # Make directory

    # ! mkdir savedVariables/{time}

    residual_array = np.array([])

    while True:
        # for iteration in [1]:

        U_previous = u_new  # store latest guess values    # change_u new to U

        A_x = A("x").createMatrix()
        A_x, b_x = boundary_conditions(A_x, b_x, U_previous, u_old, u_old_old, "x")

        ## Solve the x-equation

        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        ## Prepare matrix A and b vector for the y-equation

        A_y = A("y").createMatrix()
        A_y, b_y = boundary_conditions(A_y, b_y, U_previous, u_old, u_old_old, "y")

        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        # ## Update U.New with new x and y displacements

        u_new = np.vstack((Ux, Uy)).T

        ## Calculate the residual of each iteration

        normFactor = np.max(u_new)
        residual = math.sqrt(np.mean((u_new - U_previous) ** 2)) / normFactor
        tolerance = 1e-6

        residual_array = np.append(residual_array, residual)

        ##print out residual of each iteration
        if icorr % 100 == 0:
            moving_average = np.mean(
                residual_array[len(residual_array) - 100 : len(residual_array)]
            )
            print(
                "icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(
                    icorr, residual, normFactor, moving_average, time
                )
            )
            print(tolerance - residual)

        if icorr % 1000 == 0:

            plt.plot(np.arange(0, len(residual_array)), residual_array)
            plt.hlines(tolerance, 0, len(residual_array), color = "C1")
            plt.yscale("log")
            plt.xlabel("Iterations")
            plt.ylabel("Residual")
            plt.show()

        ## Convergence check

        if residual < tolerance:

            print(
                "icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(
                    icorr, residual, normFactor, moving_average, time
                )
            )

            break

        elif icorr > maxcorr:

            break

        icorr = icorr + 1

    ## Update variables

    u_old_old = u_old
    u_old = u_new

    # ## Add the boundary conditions to the solution matrices
    u_reshape_x = u_new[:, 0].reshape(ny + 2, nx + 2)
    u_reshape_y = u_new[:, 1].reshape(ny + 2, nx + 2)

    # #Boundary conditions for u matrix
    U = np.zeros((ny + 2, nx + 2))
    V = np.zeros((ny + 2, nx + 2))

    U[:, :] = u_reshape_x
    V[:, :] = u_reshape_y

    # Modulus of displacement

    disp_mod = np.sqrt((U ** 2) + (V ** 2))
    disp_mod_converted = disp_mod * 1e6  # Convert displacement to micrometers

    ### Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U ** 2) + (V ** 2)) * 1e-10

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])

fig.show()

print("U graph")
fig = go.Figure(data=[go.Surface(z=U, x=dispx, y=dispy)])

fig.show()

print("V graph")
fig = go.Figure(data=[go.Surface(z=V, x=dispx, y=dispy)])

fig.show()



Time = 0.0


/home/scottlevie/miniconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


icorr = 100, Residual = 0.00861212271584116916, normFactor = 0.00003249705125246536, moving average = 141922822479111.12500000000000000000, time = 0.00000
-0.00861112271584117
icorr = 200, Residual = 0.00391950110537193919, normFactor = 0.00006476581543329454, moving average = 0.07861821700023516324, time = 0.00000
-0.003918501105371939
icorr = 300, Residual = 0.00241552941255186292, normFactor = 0.00009594279739536333, moving average = 0.04010187878764360364, time = 0.00000
-0.002414529412551863
icorr = 400, Residual = 0.00169048478346032734, normFactor = 0.00012527358497624262, moving average = 0.02577807247481945022, time = 0.00000
-0.0016894847834603274
icorr = 500, Residual = 0.00126925296038366562, normFactor = 0.00015245681182245483, moving average = 0.01848121470517511550, time = 0.00000
-0.0012682529603836657


In [ ]:
U_previous

array([[ 0.00000000e+00,  0.00000000e+00],
       [-1.07247378e-05, -3.56647936e-06],
       [-3.13774960e-05, -2.54168148e-05],
       [-5.23442550e-05, -6.78773536e-05],
       [-7.28503442e-05, -1.31037246e-04],
       [-9.28121899e-05, -2.14429251e-04],
       [-1.12219431e-04, -3.17505760e-04],
       [-1.31071219e-04, -4.39711805e-04],
       [-1.49367538e-04, -5.80491914e-04],
       [-1.67108428e-04, -7.39290634e-04],
       [-1.84293929e-04, -9.15552556e-04],
       [-2.00924077e-04, -1.10872231e-03],
       [-2.16998912e-04, -1.31824456e-03],
       [-2.32518468e-04, -1.54356401e-03],
       [-2.47482784e-04, -1.78412540e-03],
       [-2.61891892e-04, -2.03937351e-03],
       [-2.75745830e-04, -2.30875315e-03],
       [-2.89044629e-04, -2.59170916e-03],
       [-3.01788322e-04, -2.88768643e-03],
       [-3.13976942e-04, -3.19612985e-03],
       [-3.25610519e-04, -3.51648438e-03],
       [-3.36689083e-04, -3.84819499e-03],
       [-3.47212662e-04, -4.19070667e-03],
       [-3.